In [27]:
import pdfplumber
import pandas as pd
import os
#import spacy 
import numpy as np
from IPython.display import clear_output
import re

In [445]:
def get_names(dirname):
    """
    Get employee names, store ids and filenames
    """
    employee_names = []
    store_ids = []
    filepaths = []
    uids = []
    for root, dirs, files in os.walk(dirname):
        for item in files:
            if ".pdf" not in item:
                skip=True
                continue
            else:
                filename = os.path.join(root, item)
                filepaths.append(filename)
                item = re.sub('[\s]+', ' ', 
                              re.sub('^\s\-\s', ' - ', 
                                     re.sub('[\(\)]+', ' - ', item)))
                item = re.sub(' - not formatted', ' ', item)
                item = re.sub('(\s\-)[A-Z0-9]', ' - ', item)
                item = re.sub('.pdf', ' ', item)
                item = re.sub('[\s]+', ' ', item)
                #id = re.findall('[A-Z0-9]{3}', item)
                splits = item.split(" - ")
                employee_name = splits[0]
                print(employee_name)
                #print(item)
                if len(splits) != 1:
                    #print(item)
                    store_id = splits[1]
                    if len(splits) == 3:
                        uid = splits[2]
                    else: 
                        uid = ''
                        print(employee_name, "don't have uid")
                    #print(store_id)
                else: 
                    store_id = ''
                    print(employee_name, "don't have store id")
                print(uid)
                employee_names.append(employee_name)
                store_ids.append(store_id) 
                uids.append(uid)
    #print("get names: finished")
    return employee_names, store_ids, filepaths, uids

In [447]:
#get_names("../Career Builder (Resumes)/")

In [448]:
def parse_text(pdf):
    """
    Parse text in pdf resume
    """
    targets = []
    education = []
    work_exp = []
    others = []
    index = dict()
    words_per_page = dict()
    for k in range(len(pdf.pages)):
        words = pdf.pages[k].extract_words(x_tolerance=5) # have a look at this hyperparam
        words_per_page[k]=len(words)
        for i in range(len(words)):
            targets.append(words[i]['text'])
        text = " ".join(targets)
    return text

In [449]:
def read_pdf(path):
    pdf = pdfplumber.open(path)
    return pdf

In [450]:
def convert_to_table(dirname):
    dirpath = dirname
    employee_names, store_ids, filepaths, uids = get_names(dirpath)
    all_dict = {"employee_name":[], "store":[], "resume":[], "uid":[]}
    useless = 0
    useless_lst = []
    for i in range(len(filepaths)):
        try:
            clear_output(wait=True)
            employee_name = employee_names[i]
            store_id = store_ids[i]
            uid = uids[i]
            path = filepaths[i]
            pdf = read_pdf(path)
            text = parse_text(pdf)
            if "Scanned by CamScanner" in text or text == "":
                useless += 1
                useless_lst.append(employee_name)
            all_dict["employee_name"].append(employee_name)
            all_dict["store"].append(store_id)
            all_dict["resume"].append(text)
            all_dict["uid"].append(uid)
            print("Current progress:", i, "/", len(filepaths)-1, np.round((i+1)/len(filepaths)*100, 2), "%")
            print("useless (scanned):", str(useless), "/", len(filepaths)-1, np.round(useless/len(filepaths)*100, 2), "%")
            print("finished", employee_name)
            print("scanned:", useless_lst)
        except TypeError:
            continue
    return pd.DataFrame(all_dict)

In [781]:
# run
dirpath = "../Career Builder (Resumes)/"
df = convert_to_table(dirpath)

Current progress: 261 / 261 100.0 %
useless (scanned): 14 / 261 5.34 %
finished Yan, Xu
scanned: ['Guilbault, Derek', 'Abualkhair, Mohd', 'Aguiar, Daniel', 'Bernard, David', 'Gandhi, Nidhi', 'Girard, Olivier', 'Gonzague, Aaron', 'Hayward, Sheldon ', 'Levesque, Vianney', 'Lirette, Mariah', 'Munguia Garcia, Rafael Antonio', 'Sukhiani, Salman', 'Tremblay, Shawnee', 'Turner, Kyle']


In [782]:
df # -> should then filter scanner 

,employee_name,store,resume,uid
0,"Abdi, Melika",TB 146,MELIKA ABDI Carmelo Avenue Ph: 6047049487 Coqu...,N3O
1,"Abdulali, Kaizar",WW 252,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-8...,BDP
2,"Abro, Muhammad Wali",WW 394,"W A ALI BRO 4 Montpelier Street, Brampton, ON ...",N62
3,"Ahmed, Ayube",WE 1248,"Ayube Ahmed Kitchener, ON ayube.ahmed@yahoo.ca...",N7Q
4,"Ahmed, Syed Furqan",TB 413,Furqan Ahmed 289-400-1879; Email: fsahmed@gmai...,
...,...,...,...,...
256,"Sukhiani, Salman",WW 341,Scanned by CamScanner Scanned by CamScanner,
257,"Tremblay, Shawnee",WE 556,Scanned by CamScanner Scanned by CamScanner Sc...,N3L
258,"Turner, Kyle",WW 854,,MPQ
259,"Wu, Alexander",WW 115,Scanned with CamScanner,


In [365]:
df.to_csv("../data/resume.csv")

In [784]:
df[df.employee_name=="Hayward, Sheldon"]

,employee_name,store,resume,uid


# Next step: join tables to get uid 

In [866]:
new_hire = pd.read_excel("../1 - New Hires for CB Resumes Pull.xlsx")
new_hire_modified = new_hire.copy()

In [867]:
resume = pd.read_csv("../data/resume.csv").drop(columns = 'Unnamed: 0')
resume.store = resume.store.astype(str)
resume.iloc[143,1] = "TB 425" # TB425 -> TB 425
resume.iloc[84,1] = "TB 416" # TB 463 -> TB 416
resume.iloc[155,1] = "WE 1168" # WW 1168 -> WE 1168
resume.iloc[182,1] = "WW 134" # WE 162 -> WW 134
resume.iloc[223,1] = "WE 502" # WE 1197 -> WE 502
resume.iloc[253,1] = "TB 701" # TB 501 -> TB 701
resume.iloc[250,1] = "TB 456" # nan -> TB 456
resume.iloc[187,0] = "Sohi, Jaipal Singh" # Sohi, Jaipal -> Sohi, Jaipal Singh

In [892]:
resume.iloc[182, :]

employee_name                                     Singh, Satwinder
store                                                       WW 134
resume           Satwinder Singh 97 Hollowgrove Blvd, 647-863-9...
uid                                                           N2V 
Name: 182, dtype: object

In [868]:
def remove_last_space(string):
    if string !="" and string[-1] == ' ':
        string = string[:-1]
    if string !="" and string[-3:] == 'FRE':
        string = string[:-4]
    return string 

In [869]:
def get_storeid(text):
    """
    Parse store_id for matching 
    """
    try:
        parse = re.match('([A-Z]{2})\s[a-zA-Z\séè\'\.\-]+([0-9]+)', text)
        banner = parse.group(1)
        code = parse.group(2)
        store_id = banner + " " + code
        return store_id
    except AttributeError:
        print(text)
        return ""

In [870]:
resume.store = resume.store.apply(lambda x: remove_last_space(x))
resume.employee_name = resume.employee_name.apply(lambda x: remove_last_space(x))

In [871]:
new_hire_modified['store'] = new_hire.Store.apply(lambda x: get_storeid(x))
new_hire_modified = new_hire_modified.drop(columns=["Store"])
new_hire_modified.rename(columns = {'Employee Name (Last, First)': "employee_name"}, inplace=True)

In [872]:
new_hire_modified.employee_name = new_hire_modified.employee_name.apply(lambda x: remove_last_space(x))

In [873]:
df_merged = (pd.merge(resume, new_hire_modified, how="left", on=["employee_name", "store"])).drop(columns=["uid", "Resume Pulled?"])

In [874]:
df_merged[["employee_name", "3-Digit ID", "resume", "store", "Hire Date", "ADP ID", "District", "Job Title Description"]]

,employee_name,3-Digit ID,resume,store,Hire Date,ADP ID,District,Job Title Description
0,"Abdi, Melika",N3O,MELIKA ABDI Carmelo Avenue Ph: 6047049487 Coqu...,TB 146,"Thursday, July 11, 2019",25237.0,BC East,SALES ASSOCIATE
1,"Abdulali, Kaizar",BDP,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-8...,WW 252,"Monday, July 29, 2019",25258.0,Alberta North,SALES MANAGER
2,"Abro, Muhammad Wali",N62,"W A ALI BRO 4 Montpelier Street, Brampton, ON ...",WW 394,"Tuesday, August 6, 2019",25331.0,GTA West,SALES ASSOCIATE
3,"Ahmed, Ayube",N7Q,"Ayube Ahmed Kitchener, ON ayube.ahmed@yahoo.ca...",WE 1248,"Monday, September 9, 2019",25403.0,ONT West - ETC,SALES ASSOCIATE
4,"Ahmed, Syed Furqan",MZQ,Furqan Ahmed 289-400-1879; Email: fsahmed@gmai...,TB 413,"Thursday, June 6, 2019",25085.0,GTA West,SALES ASSOCIATE
...,...,...,...,...,...,...,...,...
256,"Sukhiani, Salman",N03,Scanned by CamScanner Scanned by CamScanner,WW 341,"Monday, May 27, 2019",25104.0,GTA Central,SALES ASSOCIATE
257,"Tremblay, Shawnee",N3L,Scanned by CamScanner Scanned by CamScanner Sc...,WE 556,"Tuesday, July 23, 2019",25239.0,Quebec West - ETC,SALES ASSOCIATE
258,"Turner, Kyle",MPQ,NaN,WW 854,"Thursday, February 7, 2019",24774.0,Atlantic,SALES ASSOCIATE
259,"Wu, Alexander",LVW,Scanned with CamScanner,WW 115,"Thursday, May 16, 2019",25057.0,BC East,SALES ASSOCIATE


In [880]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 0 to 260
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   employee_name          261 non-null    object 
 1   store                  261 non-null    object 
 2   resume                 253 non-null    object 
 3   ADP ID                 258 non-null    float64
 4   3-Digit ID             261 non-null    object 
 5   District               259 non-null    object 
 6   Job Title Description  259 non-null    object 
 7   Hire Date              259 non-null    object 
dtypes: float64(1), object(7)
memory usage: 18.4+ KB


In [881]:
df_merged[df_merged["3-Digit ID"].isnull()]
#df_merged[df_merged.employee_name == "Engy Ibrahim"]["3-Digit ID"].isnull()

,employee_name,store,resume,ADP ID,3-Digit ID,District,Job Title Description,Hire Date


In [883]:
new_hire_modified[new_hire_modified.employee_name=="Engy Ibrahim"]

,employee_name,ADP ID,3-Digit ID,District,Job Title Description,Hire Date,Resume Pulled?,store


In [877]:
# uid doesn't exist in new_hire 
df_merged.iloc[53,4] = "MWA" # Engy Ibrahim
#df_merged.iloc[84,4] = "MW6" # Kablanian, Alexan
#df_merged.iloc[127,4] = "N1S" # Mohammed, Saifuddin
df_merged.iloc[132,0] = "Muhoza Juste"
df_merged.iloc[132,1] = "WW353"
df_merged.iloc[132,4] = "N5M"
#df_merged.iloc[187,4] = "MWN" # Sohi, Jaipal

- Engy Ibrahim and Muhoza Juste can't be found in 2019_new_hire, are they hired in 2019?
- There are 5 employee whose store recorded in 2019_new_hire is different from that recorded in the resume filename.

# finished and check then 

In [884]:
df_merged

,employee_name,store,resume,ADP ID,3-Digit ID,District,Job Title Description,Hire Date
0,"Abdi, Melika",TB 146,MELIKA ABDI Carmelo Avenue Ph: 6047049487 Coqu...,25237.0,N3O,BC East,SALES ASSOCIATE,"Thursday, July 11, 2019"
1,"Abdulali, Kaizar",WW 252,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-8...,25258.0,BDP,Alberta North,SALES MANAGER,"Monday, July 29, 2019"
2,"Abro, Muhammad Wali",WW 394,"W A ALI BRO 4 Montpelier Street, Brampton, ON ...",25331.0,N62,GTA West,SALES ASSOCIATE,"Tuesday, August 6, 2019"
3,"Ahmed, Ayube",WE 1248,"Ayube Ahmed Kitchener, ON ayube.ahmed@yahoo.ca...",25403.0,N7Q,ONT West - ETC,SALES ASSOCIATE,"Monday, September 9, 2019"
4,"Ahmed, Syed Furqan",TB 413,Furqan Ahmed 289-400-1879; Email: fsahmed@gmai...,25085.0,MZQ,GTA West,SALES ASSOCIATE,"Thursday, June 6, 2019"
...,...,...,...,...,...,...,...,...
256,"Sukhiani, Salman",WW 341,Scanned by CamScanner Scanned by CamScanner,25104.0,N03,GTA Central,SALES ASSOCIATE,"Monday, May 27, 2019"
257,"Tremblay, Shawnee",WE 556,Scanned by CamScanner Scanned by CamScanner Sc...,25239.0,N3L,Quebec West - ETC,SALES ASSOCIATE,"Tuesday, July 23, 2019"
258,"Turner, Kyle",WW 854,NaN,24774.0,MPQ,Atlantic,SALES ASSOCIATE,"Thursday, February 7, 2019"
259,"Wu, Alexander",WW 115,Scanned with CamScanner,25057.0,LVW,BC East,SALES ASSOCIATE,"Thursday, May 16, 2019"


In [889]:
len(df_merged["3-Digit ID"].unique())

261

In [891]:
scan_lst = ['Guilbault, Derek', 'Abualkhair, Mohd', 'Aguiar, Daniel', 'Bernard, David', 'Gandhi, Nidhi', 'Girard, Olivier', 'Gonzague, Aaron', 'Hayward, Sheldon ', 'Levesque, Vianney', 'Lirette, Mariah', 'Munguia Garcia, Rafael Antonio', 'Sukhiani, Salman', 'Tremblay, Shawnee', 'Turner, Kyle']
df_merged[df_merged.employee_name.isin(scan_lst)]

,employee_name,store,resume,ADP ID,3-Digit ID,District,Job Title Description,Hire Date
69,"Guilbault, Derek",WE 534,Scanned by CamScanner,19230.0,IJ4,ONT West - ETC,SALES ASSOCIATE,"Tuesday, October 15, 2019"
242,"Abualkhair, Mohd",WE 534,Scanned by CamScanner Scanned by CamScanner,25082.0,MZO,ONT West - ETC,SALES ASSOCIATE,"Tuesday, May 21, 2019"
243,"Aguiar, Daniel",TB 455,Scanned by CamScanner Scanned by CamScanner Sc...,25052.0,MYQ,Ontario West QEW,SALES ASSOCIATE,"Tuesday, May 21, 2019"
245,"Bernard, David",WE 1316,NaN,25519.0,NAV,GTA East - ETC,SALES ASSOCIATE,"Thursday, October 17, 2019"
247,"Gandhi, Nidhi",WE 526,NaN,25490.0,N9X,GTA West & Ottawa - ETC,SALES ASSOCIATE,"Tuesday, October 1, 2019"
248,"Girard, Olivier",WE 525,NaN,25565.0,NCH,Quebec West - ETC,SALES ASSOCIATE,"Monday, November 4, 2019"
249,"Gonzague, Aaron",TB 426,Scanned by CamScanner Scanned by CamScanner,25489.0,N9V,Ontario East,SALES ASSOCIATE,"Tuesday, October 1, 2019"
252,"Levesque, Vianney",WE 1127,NaN,24936.0,MVH,Quebec East & ATL - ETC,SALES ASSOCIATE,"Wednesday, April 3, 2019"
253,"Lirette, Mariah",TB 701,NaN,24817.0,MQX,Atlantic,SALES ASSOCIATE,"Sunday, February 24, 2019"
254,"Munguia Garcia, Rafael Antonio",WE 502,NaN,25010.0,MXG,Quebec West - ETC,SALES ASSOCIATE,"Wednesday, May 1, 2019"


# Summary:

- 261 employees have resumes, 14 / 261 (5.34 %) resumes are scanned (image)

---

# Then match termination 

In [905]:
df_merged2 = df_merged.rename(columns={"3-Digit ID":"Employee Code"})

In [906]:
termination = pd.read_excel("../Terminations - 2020.04.28.xlsx")

In [911]:
# join df_merged with termination 
pd.merge(df_merged2, termination, how="left", on="Employee Code")

,employee_name,store,resume,ADP ID,Employee Code,District,Job Title Description,Hire Date,AX,Seniority Date (Greater of Hire or Rehire),...,Termination Reason,Worker Category Description,Gender,Birth Year,Report Date,Tenure at Termination,Generation,Termination Type,Tenure Group - Termination,QTY
0,"Abdi, Melika",TB 146,MELIKA ABDI Carmelo Avenue Ph: 6047049487 Coqu...,25237.0,N3O,BC East,SALES ASSOCIATE,"Thursday, July 11, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Abdulali, Kaizar",WW 252,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-8...,25258.0,BDP,Alberta North,SALES MANAGER,"Monday, July 29, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Abro, Muhammad Wali",WW 394,"W A ALI BRO 4 Montpelier Street, Brampton, ON ...",25331.0,N62,GTA West,SALES ASSOCIATE,"Tuesday, August 6, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ahmed, Ayube",WE 1248,"Ayube Ahmed Kitchener, ON ayube.ahmed@yahoo.ca...",25403.0,N7Q,ONT West - ETC,SALES ASSOCIATE,"Monday, September 9, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Ahmed, Syed Furqan",TB 413,Furqan Ahmed 289-400-1879; Email: fsahmed@gmai...,25085.0,MZQ,GTA West,SALES ASSOCIATE,"Thursday, June 6, 2019",1161.0,"Thursday, June 6, 2019",...,Quit - Personal,Permanent Full Time,Male,1974.0,"Sunday, June 30, 2019",8.0,Gen X: 1965-1982,Resignation,0-3 Months,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,"Sukhiani, Salman",WW 341,Scanned by CamScanner Scanned by CamScanner,25104.0,N03,GTA Central,SALES ASSOCIATE,"Monday, May 27, 2019",1106.0,"Monday, May 27, 2019",...,Quit - Personal,Permanent Full Time,Male,1985.0,"Saturday, February 29, 2020",251.0,Millenial: 1983-1999,Resignation,6 Months - 1 Year,1.0
260,"Tremblay, Shawnee",WE 556,Scanned by CamScanner Scanned by CamScanner Sc...,25239.0,N3L,Quebec West - ETC,SALES ASSOCIATE,"Tuesday, July 23, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,"Turner, Kyle",WW 854,NaN,24774.0,MPQ,Atlantic,SALES ASSOCIATE,"Thursday, February 7, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,"Wu, Alexander",WW 115,Scanned with CamScanner,25057.0,LVW,BC East,SALES ASSOCIATE,"Thursday, May 16, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [910]:
termination[termination["Employee Code"]=="MZQ"]

,Employee Code,AX,Seniority Date (Greater of Hire or Rehire),Termination Date,Job Title,Termination Reason,Worker Category Description,Gender,Birth Year,Report Date,Tenure at Termination,Generation,Termination Type,Tenure Group - Termination,QTY
1707,MZQ,1161,"Thursday, June 6, 2019","Friday, June 14, 2019",SALES ASSOCIATE,Quit - Personal,Permanent Full Time,Male,1974,"Sunday, June 30, 2019",8,Gen X: 1965-1982,Resignation,0-3 Months,1
